# Smithy APIをBedrock AgentCore Gatewayを使ってMCPツールに変換する

## 概要
Bedrock AgentCore Gatewayは、お客様が既存のSmithy APIを完全に管理されたMCPサーバーに変換する方法を提供します。インフラストラクチャやホスティングを管理する必要はありません。お客様はSmithy仕様を持ち込み、それをmcpツールに変換できます。Amazon S3のSmithy モデルからmcpツールを作成する方法を示します。その後、エージェントはAmazon S3を照会し、それに関する質問に答えることができます。

Gatewayのワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます。
* **Gatewayのツールを作成する** - Smithy仕様を使ってツールを定義します。
* **Gatewayエンドポイントを作成する** - 受信認証付きのMCPエントリポイントとして機能するゲートウェイを作成します。
* **Gatewayにターゲットを追加する** - ゲートウェイがリクエストを特定のツールにルーティングする方法を定義するSmithy ターゲットを構成します。Smithy ファイルの一部であるすべての操作は、MCP互換のツールになり、Gateway エンドポイントURLを介して利用できるようになります。AWS IAMを使って、Smithy経由でAmazon S3 APIを呼び出すための発信認証を構成します。
* **エージェントコードを更新する** - エージェントをGatewayエンドポイントに接続して、統一されたMCPインターフェースを介してすべての構成済みツールにアクセスできるようにします。

![How does it work](images/smithy-apis-gateway.png)

### チュートリアルの詳細

| 情報                 | 詳細                                                      |
|:----------------------|:----------------------------------------------------------|
| チュートリアルの種類 | インタラクティブ                                          |
| AgentCoreコンポーネント | AgentCore Gateway、AgentCore Identity                    |
| Agenticフレームワーク | Strands Agents                                            |
| ゲートウェイターゲットの種類 | Smithy                                             |
| エージェント          | インフラストラクチャエージェント                         |
| 受信認証IdP           | Amazon Cognito                                            |
| 発信認証              | IAM                                                       |
| LLMモデル             | Anthropic Claude Sonnet 3.7、Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gatewayの作成とAgentCore Gatewayの呼び出し |
| チュートリアルの分野  | 分野横断                                                 |
| 例の複雑さ            | 簡単                                                      |
| 使用したSDK           | boto3                                                     |

チュートリアルの最初の部分では、いくつかのAmazonCore Gatewayターゲットを作成します。

### チュートリアルのアーキテクチャ
このチュートリアルでは、Amazon S3のSmithy仕様で定義された操作をMCPツールに変換し、Bedrock AgentCore Gatewayでホストします。これにより、ユーザーは自身のAWSアカウントのAmazon S3に関する質問をすることができます。

## 前提条件

このチュートリアルを実行するには、以下が必要です:
* Python 3.10+ の Jupyter notebook
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.

日本語訳:

最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
import os
# Set AWS credentials if not using SageMaker notebook

# SageMaker ノートブックを使用していない場合は、AWS 認証情報を設定します

import boto3

session = boto3.Session()
credentials = session.get_credentials()

current_credentials = session.get_credentials()
aws_access_key_id = current_credentials.access_key
aws_secret_access_key = current_credentials.secret_key

# Enter your AWS Access Key ID and AWS Secret Access Key below
# AWS アクセスキー ID と AWS シークレットアクセスキーを以下に入力してください
aws_access_key_id = ' YOUR_AWS_ACCESS_KEY_ID '
aws_secret_access_key = ' YOUR_AWS_SECRET_ACCESS_KEY '
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# Get the directory of the current script

現在のスクリプトのディレクトリを取得します。

日本語訳:

# 現在のスクリプトのディレクトリを取得する

import os

script_dir = os.path.dirname(__file__)

# script_dir には、現在実行中のスクリプトが置かれているディレクトリのパスが格納されます。
# たとえば、スクリプトが /home/user/scripts/my_script.py にある場合、
# script_dir は '/home/user/scripts' になります。

print(f"スクリプトのディレクトリ: {script_dir}")
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

__file__ が定義されていない場合の代替手段 (例: Jupyter)

# Navigate to the directory containing utils.py (one level up)

utils.py を含むディレクトリに移動します (1 レベル上)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]

日本語訳:

# sys.pathに追加する

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]
sys.path.insert(0, utils_dir)

# Now you can import utils

日本語訳:
# 今から utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role_s3_smithy("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Amazon Cognito プールを作成して Gateway への受信認証を行う

resource "aws_cognito_user_pool" "pool" {
  name = "gateway-user-pool"
}

resource "aws_cognito_user_pool_client" "client" {
  name                                 = "gateway-user-pool-client"
  user_pool_id                         = aws_cognito_user_pool.pool.id
  generate_secret                      = false
  allowed_oauth_flows_user_pool_client = true
  allowed_oauth_flows                  = ["code", "implicit"]
  callback_urls                        = ["https://example.com/oauth2/idpresponse"]
  logout_urls                          = ["https://example.com"]
}

resource "aws_apigatewayv2_authorizer" "cognito" {
  api_id           = aws_apigatewayv2_api.gateway.id
  authorizer_type  = "JWT"
  identity_sources = ["$request.header.Authorization"]
  name             = "cognito-authorizer"

  jwt_configuration {
    audience = [aws_cognito_user_pool_client.client.id]
    issuer   = "https://${aws_cognito_user_pool.pool.endpoint}"
  }
}

In [ ]:
# Creating Cognito User Pool

日本語訳:

# Cognito ユーザープールの作成

Amazon Cognito は、Web および モバイルアプリケーションの認証、承認、ユーザー管理を容易にするサービスです。ユーザープールは、ユーザーディレクトリを提供します。ユーザープールを作成するには、以下の手順に従ってください。

1. AWS マネジメントコンソールにサインインし、Cognito の画面に移動します。
2. 「ユーザープールの管理」をクリックします。
3. 「ユーザープールの作成」をクリックします。
4. ユーザープールの名前を入力します (例: `my-user-pool`)。
5. 次の手順で、必要に応じて設定を調整できます。
    - パスワードポリシーの設定
    - MFA (多要素認証) と верификация (確認) の設定
    - メッセージのカスタマイズ
    - デバイスの追跡
    - 属性の設定
    - その他の高度な設定
6. 「ユーザープールの作成」をクリックして作成を完了します。

作成したユーザープールは、ユーザーの サインアップ、サインイン、アカウント復旧などの機能を提供します。アプリケーションからは、AWS Amplify、AWS SDK、または Cognito の API を使用してユーザープールにアクセスできます。
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL

discovery_url = "https://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token"

headers = {"Metadata-Flavor": "Google"}
r = requests.get(discovery_url, headers=headers)
r.raise_for_status()
access_token = r.text

日本語訳:

# 検出 URL を取得

discovery_url = "https://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token"

headers = {"Metadata-Flavor": "Google"}
r = requests.get(discovery_url, headers=headers)
r.raise_for_status()
access_token = r.text
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# ゲートウェイを作成する

日本語訳:
# ゲートウェイを作成します

resource "aws_nat_gateway" "gw" {
  allocation_id = aws_eip.nat.id
  subnet_id     = aws_subnet.public_1.id

  tags = {
    Name = "NAT Gateway Public Subnet 1"
  }

  # To ensure proper ordering, it is recommended to add an explicit dependency
  # on the Internet Gateway for the VPC.
  depends_on = [aws_internet_gateway.gw]
}

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step

日本語訳:

# CMK を使用せずに Cognito 認証者で CreateGateway を作成します。前のステップで作成した Cognito ユーザープールを使用してください。
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g

日本語訳:
# Client は Cognito で設定された ClientId と一致する必要があります。例: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoS3Smithyv3',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールには Gateway を作成/一覧表示/取得/削除する権限が必要です。

日本語訳:
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with Smithy target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation

ゲートウェイターゲット作成に使用される GatewayID を取得します。
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Amazon S3 の仕様でSmithyターゲットを作成する

Amazon S3 APIのSmithyゲートウェイターゲットを作成します。Amazon S3 の smithy json ファイルをインポートし、API を MCP ツールに変換します。

日本語訳:

Amazon S3 の仕様で Smithy ターゲットを作成します。Amazon S3 の smithy json ファイルをインポートし、API を MCP ツールに変換します。

#### S3 Smithy API jsonファイルをS3にアップロードする

日本語訳:

S3 に S3 Smithy API json ファイルをアップロードするには、次の手順を実行します。

1. AWS マネジメントコンソールにサインインし、 S3 コンソールを開きます。
2. バケットを作成するか、既存のバケットを選択します。
3. バケットに移動し、 `アップロード` ボタンをクリックします。
4. ローカルマシンから S3 Smithy API json ファイルを選択し、 `アップロード` をクリックします。

アップロードが完了すると、S3 バケット内に S3 Smithy API json ファイルが表示されます。後続の手順でこのファイルを使用して、AWS CloudFormation を使って API Gateway REST API をデプロイできます。

In [ ]:
# Create an S3 client

日本語訳:
S3 クライアントを作成します。

import boto3

s3 = boto3.client('s3')

# Upload a file to S3
s3.upload_file('/tmp/sample_file.txt', 'my-bucket', 'sample_file.txt')

# Download a file from S3
s3.download_file('my-bucket', 'sample_file.txt', '/tmp/sample_file.txt')
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID と リージョンを取得する

account_id = subprocess.run(["aws", "sts", "get-caller-identity", "--query", "Account", "--output", "text"], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
region = subprocess.run(["aws", "configure", "get", "region"], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()

print(f"AWS アカウント ID: {account_id}")
print(f"リージョン: {region}")
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters

パラメータを定義します。

num_epochs = 10 
batch_size = 32
learning_rate = 0.001

# Import necessary libraries

必要なライブラリをインポートします。

import numpy as np
import tensorflow as tf

# Load data

データを読み込みます。

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess data

データを前処理します。

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define model

モデルを定義します。

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile model

モデルをコンパイルします。

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.

日本語訳:

# OpenAPI json ファイルをアップロードする s3 バケットを指定してください。
file_path = 'smithy-specs/s3-apis.json'
object_key = 's3-apis.json'
# ファイルをput_objectを使ってアップロードし、レスポンスを読む

日本語訳:

ファイルを put_object を使ってアップロードし、レスポンスを読む
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アカウントIDとリージョンでアップロードされたオブジェクトの ARN を構築する

日本語訳:

アカウント ID と リージョンを使用して、アップロードされたオブジェクトの Amazon リソースネーム (ARN) を構築します。技術的な用語 (ARN、アカウント ID、リージョン) はそのまま残しました。
    smithy_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {smithy_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### ゲートウェイターゲットを作成する

日本語訳:

ゲートウェイターゲットを作成するには、次の手順を実行します。

1. AWS Management Console にサインインし、 Amazon API Gateway コンソールを開きます。

2. 既存の REST API を選択するか、新しい API を作成します。

3. リソースを作成します。この例では、リソースの名前を `mydemoresource` としています。

4. `mydemoresource` リソースの下に新しい子リソースを作成します。この例では、子リソースの名前を `mydemoresourcechild` としています。

5. `mydemoresourcechild` リソースに対して、HTTP メソッド要求を設定します。この例では、 GET メソッドを選択しています。

6. 統合タイプとして、「 Lambda 関数」を選択します。

7. Lambda 関数を選択するか、新しい関数を作成します。この例では、既存の関数 `lambda_function_name` を選択しています。

8. 必要に応じて、その他の設定を行います。

9. API Gateway コンソールで、「リソースポリシーの追加」を選択し、リソースポリシーを設定します。この例では、ワイルドカード (`/*`) を使用しています。

10. 「デプロイ」を選択し、ステージ名 (例: `prod`) を入力して、API をデプロイします。

これで、 API Gateway 経由で Lambda 関数を呼び出すことができるようになりました。呼び出し URL は、 `https://{restapi_id}.execute-api.{region}.amazonaws.com/prod/mydemoresource/mydemoresourcechild` のようになります。

In [ ]:
# S3 Uri for Smithy spec file

日本語訳:

# Smithy 仕様ファイルの S3 URI
smithy_s3_target_config = {
    "mcp": {
          "smithyModel": {
              "s3": {
                  "uri": smithy_s3_uri
              }
          }
      }
}

# IAM 認証情報プロバイダーの設定

日本語訳:
credential_config  = {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }

targetname='DemoSmithytargetForS3'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Smithy Target with S3Uri using SDK',
    targetConfiguration=smithy_s3_target_config,
    credentialProviderConfigurations=[credential_config])

# リクエスト ID とタイムスタンプを出力します。不具合/欠陥を報告する際にこれらを含めてください。

日本語訳:

リクエスト ID とタイムスタンプを出力しています。不具合/欠陥を報告する際には、これらを含めてください。
response_metadata = response['ResponseMetadata']
print(response_metadata)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて、AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway の中核は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠の Gateway です。これらの API を使用すると、MCP 準拠のクライアントまたは SDK から、利用可能なツールを安全で標準化された方法で検出およびやり取りできます。Strands エージェントが AWS サービスにアクセスする必要がある場合は、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は、(MCP Authorization 仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] を厳密に順守しており、堅牢なセキュリティとアクセス制御が保証されています。つまり、Strands エージェントによるすべてのツール呼び出しは、セキュリティを維持しながら強力な機能を可能にする認証ステップを経ます。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを検出し、次に InvokeTools を使用して特定のアクションを実行します。Gateway は、必要なすべてのセキュリティ検証、プロトコル変換、およびサービス対話を処理するため、プロセス全体がシームレスかつ安全に行われます。

このアーキテクチャアプローチにより、MCP 仕様を実装するクライアントまたは SDK は、Gateway を介して AWS サービスとやり取りできるため、AI エージェントの統合に対して汎用性と将来性のある解決策となります。

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# Amazon Cognitoからアクセストークンを要求して、インバウンド認証を行う

日本語訳:

 access_token = aws_auth.get_access_token()  # アクセストークンを取得

 operation = 'PutItem'  # 実行する操作
 table_name = 'Employees'  # テーブル名
 key = {'EmpId': {'N': ' 123 '}}  # パーティションキー

 attribute_values = {
     ':name': {'S': ' Jane Doe '},
     ':job': {'S': ' Engineer '},
     ':city': {'S': ' Seattle '}
 }

 condition_expression = 'attribute_not_exists(EmpId)'  # 条件式

 # DynamoDBにアイテムを追加するリクエスト
 response = dynamodb.meta.client.put_item(
     TableName=table_name,
     Item={
         'EmpId': key['EmpId'],
         'Name': attribute_values[':name'],
         'JobTitle': attribute_values[':job'],
         'City': attribute_values[':city']
     },
     ConditionExpression=condition_expression
 )

 print(response)

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# AWS アカウントで保有している s3 リソースについて IT エージェントに尋ねる

日本語訳:
あなたの AWS アカウントで保有している s3 リソースについて IT エージェントに尋ねてください。

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

以下が日本語訳になります。

## ~/.aws/credentials に設定された IAM グループ/ユーザーは Bedrock モデルへのアクセス権限を持つ必要があります
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.

日本語訳:

# ルート strands ロガーを設定します。問題をデバッグしている場合は、これを DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 7
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 

listTools を呼び出します。
    tools = client.list_tools_sync()
    # エージェントを model と tools で作成する

日本語訳:

# エージェントを model と tools で作成する

agent = Agent(model, tools)
    agent = Agent(model=yourmodel,tools=tools) ## お好みの モデル に置き換えることができます
    #print(f"Tools loaded in the agent are {agent.tool_names}")

日本語訳:

エージェントに読み込まれたツールは { agent.tool_names } です
    #print(f"Tools configuration in the agent are {agent.tool_config}")

エージェントのツール設定は {agent.tool_config} です。
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.

サンプルのプロンプトでエージェントを呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下では実際にはツールを呼び出しません。
    agent("Hi , can you list all the s3 buckets. Locate the right tool and call the tool to get the answer")
    #agent("What is the weather in northern part of the mars")

日本語訳:

#agent("火星の北部の天気はどうですか")
    # Invoke the agent with sample prompt, invoke the tool and display the response

サンプルのプロンプトでエージェントを呼び出し、ツールを呼び出して、レスポンスを表示します。
    # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 70
    # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 71
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 72
    name=targetname+"___ListBuckets", # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 73
    # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 74
    )
    # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs) # 出力: <Object logs>

# 半角英数字の前後に半角スペースを挿入
run.info('42 is the answer') # 出力: 42 is the answer
run.info('sum = 6 * 7') # 出力: sum = 6 * 75
    print(f"Tool Call result: {result['content'][0]['text']}")


# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、s3 バケットなどの追加リソースも作成されます。これらは、実行した例次第で、クリーンアップの一環として手動で削除する必要があるかもしれません。

## ゲートウェイを削除する (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)